# Building a simple agent framework

## LLM Wrapper

In [16]:
import os
import json
import time
import traceback
from litellm import completion
from dotenv import load_dotenv
from dataclasses import dataclass, field
from collections.abc import Callable
from typing import Any, Union


load_dotenv()


@dataclass
class Prompt:
    messages: list[dict] = field(default_factory=list)
    tools: list[dict] = field(default_factory=list)
    metadata: dict = field(default_factory=dict)


def generate_response(prompt: Prompt) -> str:
    """Call LLM to get response"""

    messages = prompt.messages
    tools = prompt.tools

    if not tools:
        response = completion(
            model="openai/gpt-4o",
            messages=messages,
            max_tokens=1024
        )
        return response.choices[0].message.content

    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        tools=tools,
        max_tokens=1024
    )

    if response.choices[0].message.tool_calls:
        tool = response.choices[0].message.tool_calls[0]
        result = {
            "tool": tool.function.name,
            "args": json.loads(tool.function.arguments),
        }
        return json.dumps(result)

    return response.choices[0].message.content

## GAME Components

In [17]:
@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str


class Action:
    def __init__(self,
                 name: str,
                 function: Callable,
                 description: str,
                 parameters: dict,
                 terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.terminal = terminal
        self.parameters = parameters

    def execute(self, **args) -> Any:
        """Execute the action's function"""
        return self.function(**args)


class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str) -> Union[Action, None]:
        return self.actions.get(name, None)

    def get_actions(self) -> list[Action]:
        """Get all registered actions"""
        return list(self.actions.values())


class Memory:
    def __init__(self):
        self.items = []  # Basic conversation history

    def add_memory(self, memory: dict):
        """Add memory to working memory"""
        self.items.append(memory)

    def get_memories(self, limit: int = None) -> list[dict]:
        """Get formatted conversation history for prompt"""
        return self.items[:limit]

    def copy_without_system_memories(self):
        """Return a copy of the memory without system memories"""
        filtered_items = [m for m in self.items if m["type"] != "system"]
        memory = Memory()
        memory.items = filtered_items
        return memory


class Environment:
    def execute_action(self, action: Action, args: dict) -> dict:
        """Execute an action and return the result."""
        try:
            result = action.execute(**args)
            return self.format_result(result)
        except Exception as e:
            return {
                "tool_executed": False,
                "error": str(e),
                "traceback": traceback.format_exc()
            }

    @staticmethod
    def format_result(result: Any) -> dict:
        """Format the result with metadata."""
        return {
            "tool_executed": True,
            "result": result,
            "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S%z")
        }

## Agent Languages

In [19]:
class AgentLanguage:
    """
    The AgentLanguage component has two primary responsibilities:
    - Prompt Construction: Transforming our GAME components into a format the LLM can understand
    - Response Parsing: Interpreting the LLM’s response to determine what action the agent should take
    """
    def construct_prompt(self, actions: list[Action], environment: Environment, goal_list: list[Goal], memory: Memory) -> Prompt:
        raise NotImplementedError("Subclasses must implement this method")

    def parse_response(self, response: str) -> dict:
        raise NotImplementedError("Subclasses must implement this method")

In [20]:
class AgentFunctionCallingActionLanguage(AgentLanguage):
    def construct_prompt(self, actions: list[Action], environment: Environment, goal_list: list[Goal], memory: Memory) -> Prompt:

        prompt =  self.format_goals(goal_list) + self.format_memory(memory)
        tools = self.format_actions(actions)

        return Prompt(messages=prompt, tools=tools)

    def parse_response(self, response: str) -> dict:
        """Parse LLM response into structured format by extracting the ```json block"""
        try:
            return json.loads(response)

        except Exception:
            return {
                "tool": "terminate",
                "args": {"message": response}
            }

    @staticmethod
    def format_goals(goal_list: list[Goal]) -> list:
        # Map all goals to a single string that concatenates their description
        # and combine into a single message of type system
        sep = "\n-------------------\n"
        goal_instructions = "\n\n".join([f"{goal.name}:{sep}{goal.description}{sep}" for goal in goal_list])
        return [
            {"role": "system", "content": goal_instructions}
        ]

    @staticmethod
    def format_memory(memory: Memory) -> list:
        """Generate response from language model"""
        # Map all environment results to a role:user messages
        # Map all assistant messages to a role:assistant messages
        # Map all user messages to a role:user messages
        items = memory.get_memories()
        mapped_items = []
        for item in items:

            content = item.get("content", None)
            if not content:
                content = json.dumps(item, indent=4)

            if item["type"] == "assistant":
                mapped_items.append({"role": "assistant", "content": content})
            elif item["type"] == "environment":
                mapped_items.append({"role": "assistant", "content": content})
            else:
                mapped_items.append({"role": "user", "content": content})

        return mapped_items

    @staticmethod
    def format_actions(actions: list[Action]) -> list[dict]:
        """Generate response from language model"""
        tools = [
            {
                "type": "function",
                "function": {
                    "name": action.name,
                    # Include up to 1024 characters of the description
                    "description": action.description[:1024],
                    "parameters": action.parameters,
                },
            } for action in actions
        ]
        return tools

    @staticmethod
    def adapt_prompt_after_parsing_error(prompt: Prompt, response: str, traceback: str, error: Any, retries_left: int) -> Prompt:
        return prompt


class AgentJsonActionLanguage(AgentLanguage):
    action_format = """
<Stop and think step by step. Insert your thoughts here.>

```action
{
    "tool": "tool_name",
    "args": {...fill in arguments...}
}
```"""

    def format_actions(self, actions: list[Action]) -> list:
        # Convert actions to a description the LLM can understand
        action_descriptions = [
            {
                "name": action.name,
                "description": action.description,
                "args": action.parameters
            }
            for action in actions
        ]

        return [{
            "role": "system",
            "content": f"""
Available Tools: {json.dumps(action_descriptions, indent=4)}

{self.action_format}
"""
        }]

    def parse_response(self, response: str) -> dict:
        """Extract and parse the action block"""
        try:
            start_marker = "```action"
            end_marker = "```"

            stripped_response = response.strip()
            start_index = stripped_response.find(start_marker)
            end_index = stripped_response.rfind(end_marker)
            json_str = stripped_response[
                start_index + len(start_marker):end_index
            ].strip()

            return json.loads(json_str)
        except Exception as e:
            print(f"Failed to parse response: {str(e)}")
            raise e


## Agent

In [21]:
class Agent:
    def __init__(self, goal_list: list[Goal], agent_language: AgentLanguage, action_registry: ActionRegistry,
                 generate_response: Callable[[Prompt], str], environment: Environment):
        """
        Initialize an agent with its core GAME components
        """
        self.goals = goal_list
        self.generate_response = generate_response
        self.agent_language = agent_language
        self.actions = action_registry
        self.environment = environment

    def construct_prompt(self, goal_list: list[Goal], memory: Memory, actions: ActionRegistry) -> Prompt:
        """Build prompt with memory context"""
        return self.agent_language.construct_prompt(
            actions=actions.get_actions(),
            environment=self.environment,
            goal_list=goal_list,
            memory=memory
        )

    def get_action(self, response) -> tuple[Action, dict]:
        invocation = self.agent_language.parse_response(response)
        action = self.actions.get_action(invocation["tool"])
        return action, invocation

    def should_terminate(self, response: str) -> bool:
        action_def, _ = self.get_action(response)
        return action_def.terminal

    def set_current_task(self, memory: Memory, task: str):
        memory.add_memory({"type": "user", "content": task})

    def update_memory(self, memory: Memory, response: str, result: dict):
        """
        Update memory with the agent's decision and the environment's response.
        """
        new_memories = [
            {"type": "assistant", "content": response},
            {"type": "environment", "content": json.dumps(result)}
        ]
        for m in new_memories:
            memory.add_memory(m)

    def prompt_llm_for_action(self, full_prompt: Prompt) -> str:
        response = self.generate_response(full_prompt)
        return response

    def run(self, user_input: str, memory=None, max_iterations: int = 50) -> Memory:
        """
        Execute the GAME loop for this agent with a maximum iteration limit.
        """
        memory = memory or Memory()
        self.set_current_task(memory, user_input)

        for _ in range(max_iterations):
            # Construct a prompt that includes the Goals, Actions, and the current Memory
            prompt = self.construct_prompt(self.goals, memory, self.actions)

            print("Agent thinking...")
            # Generate a response from the agent
            response = self.prompt_llm_for_action(prompt)
            print(f"Agent Decision: {response}")

            # Determine which action the agent wants to execute
            action, invocation = self.get_action(response)

            # Execute the action in the environment
            result = self.environment.execute_action(action, invocation["args"])
            print(f"Action Result: {result}")

            # Update the agent's memory with information about what happened
            self.update_memory(memory, response, result)

            # Check if the agent has decided to terminate
            if self.should_terminate(response):
                break

        return memory


## Register Actions

In [22]:
def read_project_file(name: str) -> str:
    with open(name, "r") as f:
        return f.read()

def list_project_files() -> list[str]:
    return sorted([file for file in os.listdir(".") if file.endswith(".py")])


# Define the action registry and register some actions
action_registry = ActionRegistry()
action_registry.register(Action(
    name="list_project_files",
    function=list_project_files,
    description="Lists all files in the project.",
    parameters={},
    terminal=False
))
action_registry.register(Action(
    name="read_project_file",
    function=read_project_file,
    description="Reads a file from the project.",
    parameters={
        "type": "object",
        "properties": {
            "name": {"type": "string"}
        },
        "required": ["name"]
    },
    terminal=False
))
action_registry.register(Action(
    name="terminate",
    function=lambda message: f"{message}\nTerminating...",
    description="Terminates the session and prints the message to the user.",
    parameters={
        "type": "object",
        "properties": {
            "message": {"type": "string"}
        },
        "required": []
    },
    terminal=True
))

## Main

In [23]:
# Define the agent's goals
goals = [
    Goal(priority=1, name="Gather Information", description="Read each file in the project"),
    Goal(priority=1, name="Terminate", description="Call the terminate call when you have read all the files "
                                                   "and provide the content of the README in the terminate message")
]

# Define the agent's language
agt_language = AgentFunctionCallingActionLanguage()

# Define the environment
exec_environment = Environment()

# Create an agent instance
agent = Agent(goals, agt_language, action_registry, generate_response, exec_environment)

# Run the agent with user input
user_instructions = "Write a README for this project."
final_memory = agent.run(user_instructions)

# Print the final memory
print(final_memory.get_memories())

Agent thinking...
Agent Decision: {"tool": "list_project_files", "args": {}}
Action Result: {'tool_executed': True, 'result': ['ztp_xr_custom.py'], 'timestamp': '2025-12-18T22:06:58-0500'}
Agent thinking...
Agent Decision: {"tool": "read_project_file", "args": {"name": "ztp_xr_custom.py"}}
Action Result: {'tool_executed': True, 'result': '#! /usr/bin/env python\n"""\n ztp_xr_custom\n\n Copyright (c) 2020 Cisco Systems, Inc. and/or its affiliates\n @author Marcelo Reis\n @version 1.9, 27/11/2020\n"""\nimport sys\nimport os\nimport logging\nimport json\nimport re\nimport urlparse\nimport urllib2\nimport socket\nimport base64\nimport time\nfrom functools import partial\n\nsys.path.append("/pkg/bin/")\nfrom ztp_helper import ZtpHelpers\n\nMETADATA_URL = \'http://192.168.122.211/ztp_metadata.json\'\nSYSLOG_CONFIG = {\n    \'syslog_file\': \'/disk0:/ztp/ztp_python.log\',\n    \'syslog_server\': \'192.168.122.211\',\n    \'syslog_port\': 514\n}\n\n\ndef main():\n    ztp_api = ZtpApi(**SYSLOG_